In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]

In [132]:
textContent = {}

for WikiPage in AnimalsWikiPages:
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):
        
        # Skip useless stuff
        if Tags.span == None:
            continue
        
        # Set chapter variable    
        Chapter = Tags.span.attrs['id']
        
        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':
            
            
            # Get species name
            Species = soup.title\
                            .string.split('-')[0]\
                            .rstrip(' ')
            # Create list for key
            textContent[Species] = []

            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):
                # But only if chapter is still valid as previous sibling
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    textContent[Species].append(Text.text)